# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
from api_keys import weather_api_key

In [ ]:
# output_data_file = "starter_code/city_df.csv"
# cityDF = pd.read_csv(output_data_file)
# ^^ not sure why this wasnt working ^^

url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"
query_url = f"{url}appid={weather_api_key}&units={units}&q="

city_list = ["Rome", "Paris", "London", "Miami", "Sur",  'airai', 'victoria',
 'faanui','jiayuguan','kodiak','kruisfontein','ushuaia', 'kharp',  'busselton','yellowknife',
 'mata','tuktoyaktuk','stornoway','arraial do cabo','guerrero negro','pavlovsk','saldanha','bluff',
 'makakilo city','lere','cape town','astana','grindavik','amstetten',
 "Newark", "Chicago", "Florence", "Nice", "Toronto", "Accra", "Boke", "Cape Town", "Buenos Aires", "Lima",
 "Cartagena", "Mumbai", "Kabul", "Karachi", "Hanoi", "Kuala Lumpur", "Jakarta", "Manila", "Wellington",
 "Sydney", "Melbourne", "Auckland", "Suva", "Anchorage", "Moscow", "Helsinki", "Tallinn"]

# had to create my own list because when I was using the varibale 'cities' defined above it was not working
# and the tutor couldnt figure out why

lat = []
lng = []
temp = []
cloud = []
wind_speed = []
hum = []

for city in city_list:
    response = requests.get(query_url + city).json()
    lat.append(response["coord"]['lat'])
    lng.append(response["coord"]['lon'])
    temp.append(response["main"]['temp']*1.8+32)
    cloud.append(response["clouds"]["all"])
    wind_speed.append(response["wind"]["speed"])
    hum.append(response["main"]["humidity"])
    
city_df = pd.DataFrame({
    "city": city_list,
    "latitude":lat,
    "longitude": lng,
    "temp": temp,
    "cloudiness": cloud,
    "wind speed": wind_speed,
    "humidity":  hum
})
city_df.to_csv("city_df.csv")
city_df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
locations = city_df[["latitude", "longitude"]]
humidity = city_df["humidity"]

In [ ]:
fig = gmaps.figure()

# Create heat layer

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=10,
                                point_radius=1)

# add layer

fig.add_layer(heat_layer)

#display figure
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
vac = city_df[city_df["temp"] > 70]
hotel_df = vac[vac["humidity"] < 75]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df["hotel name"] = ""
hotel_df

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    city_name = row['city']
    lat = row["latitude"]
    lng = row["longitude"]
    # add keyword to params dict
    params['location']= f'{lat},{lng}'

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    #pprint(response)
    hotel_df.loc[index,'hotel name'] = response["results"][0]['name']



In [ ]:
hotel_df.dropna(inplace=True)
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [ ]:
# Add marker layer ontop of heat map
locations = hotel_df[["latitude", "longitude"]]
hotels = hotel_df["humidity"]

# Display figure
fig = gmaps.figure()

# Create heat layer

heat_layer = gmaps.heatmap_layer(locations, weights=hotels,
                                dissipating=False, max_intensity=10,
                                point_radius=1)

# add layer

fig.add_layer(heat_layer)

#display figure
fig